<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies_noncovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 5.6MB 5.6MB/s 
     |████████████████████████████████| 184kB 48.7MB/s 
     |████████████████████████████████| 3.2MB 50.3MB/s 
ERROR: requests 2.23.0 has requirement chardet<4,>=3.0.2, but you'll have chardet 4.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 14


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210414 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210414 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [19]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [20]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [21]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [22]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v_nc > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v_nc > 0) & \
    (df_cap.icu_o_nc > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 
                              'icu_v_nc', 'nonicu_v_nc', 'mechvent_v_nc', 
                              'icu_o_nc', 'nonicu_o_nc', 'mechvent_o_nc',
                              'Landline Number', 'Landline Number 2', 'Fax Number', 
                              'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v_nc', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [23]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,320255,2021-04-11,EAST AVENUE MEDICAL CENTER,QUEZON CITY,68,96,14,34,199,13,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
1,306594,2021-04-11,PHILIPPINE HEART CENTER,QUEZON CITY,51,36,57,5,43,9,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,192081,2021-04-11,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,49,186,22,13,95,17,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,178130,2021-04-11,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,39,32,3,14,65,3,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,329313,2021-04-11,NATIONAL KIDNEY AND TRANSPLANT INSTITUTE,QUEZON CITY,37,37,13,81,81,7,(02) 8981 0330,(02) 8924 0701,(02) 8922 5608,Government
5,6279,2021-04-11,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,31,47,7,4,52,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
6,156096,2021-04-11,AMANG RODRIGUEZ MEMORIAL MEDICAL CENTER,CITY OF MARIKINA,30,149,11,19,50,8,(02) 8942 0245,None,None,Government
7,107913,2021-04-11,OUR LADY OF LOURDES HOSPITAL,CITY OF MANILA,29,96,8,5,31,0,(02) 8716 8001,(02) 8252 3942,(02) 8714 7153,Private
8,258402,2021-04-11,PASAY CITY GENERAL HOSPITAL,PASAY CITY,24,47,6,10,26,3,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
9,9421,2021-04-11,"WORLD CITI MEDICAL CENTER, INC.",QUEZON CITY,21,25,1,3,22,2,(02) 8990 9262,(02) 3438 4585,(02) 8913 8383,Private


In [24]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,320256,2021-04-12,EAST AVENUE MEDICAL CENTER,QUEZON CITY,69,83,12,30,204,10,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
1,306595,2021-04-12,PHILIPPINE HEART CENTER,QUEZON CITY,52,34,56,5,45,10,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,192082,2021-04-12,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,49,191,28,13,90,23,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,178131,2021-04-12,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,41,19,4,12,78,2,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,6280,2021-04-12,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,31,38,6,4,61,1,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,283353,2021-04-12,"MEDICAL CENTER OF TAGUIG CITY, INC.",TAGUIG CITY,1,13,2,2,12,0,(02) 8888 6284,(02) 8399 0494,(02) 8888 6284,Private
57,296971,2021-04-12,"ST. CLARE?S MEDICAL CENTER, INC.",CITY OF MAKATI,1,28,1,1,9,1,(02) 8831 6511,(02) 8831 6512,(02) 8831 5445,Private
58,300335,2021-04-12,"MEDICAL CENTER MANILA, INC.",ERMITA,1,74,9,4,38,4,(02) 8523 8131,(02) 8522 3799,(02) 8524 3440,Private
59,46936,2021-04-12,DR. JESUS C. DELGADO MEMORIAL HOSPITAL,QUEZON CITY,1,39,2,1,8,1,(02) 8924 4051,(02) 8925 2991,(02) 8928 5945,Private


In [25]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,320257,2021-04-13,EAST AVENUE MEDICAL CENTER,QUEZON CITY,72,79,38,31,210,10,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
1,306596,2021-04-13,PHILIPPINE HEART CENTER,QUEZON CITY,49,36,56,11,43,10,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,192083,2021-04-13,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,48,190,22,14,91,20,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,178132,2021-04-13,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,39,16,3,14,81,3,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,6281,2021-04-13,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,31,43,7,4,56,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,46937,2021-04-13,DR. JESUS C. DELGADO MEMORIAL HOSPITAL,QUEZON CITY,1,40,1,1,7,2,(02) 8924 4051,(02) 8925 2991,(02) 8928 5945,Private
59,245862,2021-04-13,PASIG DOCTORS MEDICAL CENTER INC.,CITY OF PASIG,1,27,1,2,3,2,(02) 8997 2791,(02) 8878 7362,None,Private
60,62927,2021-04-13,NATIONAL CHILDREN?S HOSPITAL,QUEZON CITY,1,26,6,3,3,3,(02) 8724 0656,None,(02) 8721 9139,Government
61,155241,2021-04-13,UNIHEALTH PARA?AQUE HOSPITAL AND MEDICAL CENTE...,CITY OF PARAÑAQUE,1,17,2,3,19,4,(02) 8832 0636,(02) 8832 0641,(02) 8832 0627,Private


In [26]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o_nc'].sum()
vacant = df_cap[filter]['icu_v_nc'].sum()
pct_used = (occupied / (vacant + occupied)) * 100